In [1]:
import $ivy.`org.apache.spark::spark-sql:3.5.0`
import $ivy.`org.postgresql:postgresql:42.7.2`
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import $ivy.$
import $ivy.$
import org.apache.log4j.{Level, Logger}

In [2]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions._

import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions._

In [3]:
val spark = SparkSession
                .builder()
                .master("local[*]")
                .appName("JDBC Data Source")
                .config("spark.driver.memory", "16g")
                .getOrCreate()

import spark.implicits._

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
24/03/13 20:48:13 INFO SparkContext: Running Spark version 3.5.0
24/03/13 20:48:13 INFO SparkContext: OS info Linux, 6.5.0-25-generic, amd64
24/03/13 20:48:13 INFO SparkContext: Java version 11.0.22
24/03/13 20:48:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/13 20:48:14 INFO ResourceUtils: ==============================================================
24/03/13 20:48:14 INFO ResourceUtils: No custom resources configured for spark.driver.
24/03/13 20:48:14 INFO ResourceUtils: ==============================================================
24/03/13 20:48:14 INFO SparkContext: Submitted application: JDBC Data Source
24/03/13 20:48:14 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 1, script: , vendor: , memory -> name: memory, amount: 1024, script: , vendor: , off

spark: SparkSession = org.apache.spark.sql.SparkSession@780d647b
import spark.implicits._

In [4]:
val driver = "org.postgresql.Driver"
val url = "jdbc:postgresql://localhost:5432/spark"
val user = "postgres"
val password = "postgres"

driver: String = "org.postgresql.Driver"
url: String = "jdbc:postgresql://localhost:5432/spark"
user: String = "postgres"
password: String = "postgres"

### Партиционирование по столбцам

In [5]:
val df102 = spark.read
    .format("jdbc")
    .option("driver", driver)
    .option("url", url)
    .option("user", user)
    .option("password", password)
    .option("dbtable", "public.employees")
    .option("partitionColumn", "emp_no")
    .option("lowerBound", "10010")
    .option("upperBound", "499990")
    .option("numPartitions", "10")
    .load()

24/03/13 20:48:16 INFO SharedState: Setting hive.metastore.warehouse.dir ('null') to the value of spark.sql.warehouse.dir.
24/03/13 20:48:16 INFO SharedState: Warehouse path is 'file:/home/vadim/workspace/Spark/JDBC/spark-warehouse'.
24/03/13 20:48:17 INFO JDBCRelation: Number of partitions: 10, WHERE clauses of these partitions: "emp_no" < 59008 or "emp_no" is null, "emp_no" >= 59008 AND "emp_no" < 108006, "emp_no" >= 108006 AND "emp_no" < 157004, "emp_no" >= 157004 AND "emp_no" < 206002, "emp_no" >= 206002 AND "emp_no" < 255000, "emp_no" >= 255000 AND "emp_no" < 303998, "emp_no" >= 303998 AND "emp_no" < 352996, "emp_no" >= 352996 AND "emp_no" < 401994, "emp_no" >= 401994 AND "emp_no" < 450992, "emp_no" >= 450992


df102: org.apache.spark.sql.package.DataFrame = [emp_no: int, birth_date: date ... 4 more fields]

Number of partitions: 10

WHERE clauses of these partitions: 
- "emp_no" < 59008 or "emp_no" is null
- "emp_no" >= 59008 AND "emp_no" < 108006
- "emp_no" >= 108006 AND "emp_no" < 157004
- "emp_no" >= 157004 AND "emp_no" < 206002
- "emp_no" >= 206002 AND "emp_no" < 255000
- "emp_no" >= 255000 AND "emp_no" < 303998
- "emp_no" >= 303998 AND "emp_no" < 352996
- "emp_no" >= 352996 AND "emp_no" < 401994
- "emp_no" >= 401994 AND "emp_no" < 450992
- "emp_no" >= 450992

In [6]:
println(s"count = ${df102.count()}\nnum partitions = ${df102.rdd.getNumPartitions}")

24/03/13 20:48:18 INFO CodeGenerator: Code generated in 160.106895 ms
24/03/13 20:48:19 INFO DAGScheduler: Registering RDD 2 (count at cell6.sc:1) as input to shuffle 0
24/03/13 20:48:19 INFO DAGScheduler: Got map stage job 0 (count at cell6.sc:1) with 10 output partitions
24/03/13 20:48:19 INFO DAGScheduler: Final stage: ShuffleMapStage 0 (count at cell6.sc:1)
24/03/13 20:48:19 INFO DAGScheduler: Parents of final stage: List()
24/03/13 20:48:19 INFO DAGScheduler: Missing parents: List()
24/03/13 20:48:19 INFO DAGScheduler: Submitting ShuffleMapStage 0 (MapPartitionsRDD[2] at count at cell6.sc:1), which has no missing parents
24/03/13 20:48:19 INFO MemoryStore: Block broadcast_0 stored as values in memory (estimated size 15.2 KiB, free 4.5 GiB)
24/03/13 20:48:19 INFO MemoryStore: Block broadcast_0_piece0 stored as bytes in memory (estimated size 7.7 KiB, free 4.5 GiB)
24/03/13 20:48:19 INFO BlockManagerInfo: Added broadcast_0_piece0 in memory on ubuntu:45869 (size: 7.7 KiB, free: 4.5 G

count = 30003
num partitions = 10


In [7]:
df102.rdd.mapPartitionsWithIndex { (p,i) => List((p, i.length)).iterator }.collect()

24/03/13 20:48:20 INFO SparkContext: Starting job: collect at cell7.sc:1
24/03/13 20:48:20 INFO DAGScheduler: Got job 2 (collect at cell7.sc:1) with 10 output partitions
24/03/13 20:48:20 INFO DAGScheduler: Final stage: ResultStage 3 (collect at cell7.sc:1)
24/03/13 20:48:20 INFO DAGScheduler: Parents of final stage: List()
24/03/13 20:48:20 INFO DAGScheduler: Missing parents: List()
24/03/13 20:48:20 INFO DAGScheduler: Submitting ResultStage 3 (MapPartitionsRDD[11] at mapPartitionsWithIndex at cell7.sc:1), which has no missing parents
24/03/13 20:48:20 INFO MemoryStore: Block broadcast_2 stored as values in memory (estimated size 21.9 KiB, free 4.5 GiB)
24/03/13 20:48:20 INFO MemoryStore: Block broadcast_2_piece0 stored as bytes in memory (estimated size 9.9 KiB, free 4.5 GiB)
24/03/13 20:48:20 INFO BlockManagerInfo: Added broadcast_2_piece0 in memory on ubuntu:45869 (size: 9.9 KiB, free: 4.5 GiB)
24/03/13 20:48:20 INFO SparkContext: Created broadcast 2 from broadcast at DAGScheduler.

res7: Array[(Int, Int)] = Array(
  (0, 4900),
  (1, 4900),
  (2, 203),
  (3, 601),
  (4, 4899),
  (5, 4500),
  (6, 0),
  (7, 200),
  (8, 4900),
  (9, 4900)
)

Зададим в качестве *lowerBound* и *upperBound* произвольные значения (не min и max)

In [8]:
val df103 = spark.read
    .format("jdbc")
    .option("driver", driver)
    .option("url", url)
    .option("user", user)
    .option("password", password)
    .option("dbtable", "public.employees")
    .option("partitionColumn", "emp_no")
    .option("lowerBound", "20000")
    .option("upperBound", "50000")
    .option("numPartitions", "10")
    .load()

24/03/13 20:48:21 INFO JDBCRelation: Number of partitions: 10, WHERE clauses of these partitions: "emp_no" < 23000 or "emp_no" is null, "emp_no" >= 23000 AND "emp_no" < 26000, "emp_no" >= 26000 AND "emp_no" < 29000, "emp_no" >= 29000 AND "emp_no" < 32000, "emp_no" >= 32000 AND "emp_no" < 35000, "emp_no" >= 35000 AND "emp_no" < 38000, "emp_no" >= 38000 AND "emp_no" < 41000, "emp_no" >= 41000 AND "emp_no" < 44000, "emp_no" >= 44000 AND "emp_no" < 47000, "emp_no" >= 47000


df103: org.apache.spark.sql.package.DataFrame = [emp_no: int, birth_date: date ... 4 more fields]

Number of partitions: 10

WHERE clauses of these partitions:
- "emp_no" < 23000 or "emp_no" is null
- "emp_no" >= 23000 AND "emp_no" < 26000
- "emp_no" >= 26000 AND "emp_no" < 29000
- "emp_no" >= 29000 AND "emp_no" < 32000
- "emp_no" >= 32000 AND "emp_no" < 35000
- "emp_no" >= 35000 AND "emp_no" < 38000
- "emp_no" >= 38000 AND "emp_no" < 41000
- "emp_no" >= 41000 AND "emp_no" < 44000
- "emp_no" >= 44000 AND "emp_no" < 47000
- "emp_no" >= 47000

In [9]:
println(s"count = ${df103.count()}\nnum partitions = ${df103.rdd.getNumPartitions}")

24/03/13 20:48:22 INFO DAGScheduler: Registering RDD 14 (count at cell9.sc:1) as input to shuffle 1
24/03/13 20:48:22 INFO DAGScheduler: Got map stage job 3 (count at cell9.sc:1) with 10 output partitions
24/03/13 20:48:22 INFO DAGScheduler: Final stage: ShuffleMapStage 4 (count at cell9.sc:1)
24/03/13 20:48:22 INFO DAGScheduler: Parents of final stage: List()
24/03/13 20:48:22 INFO DAGScheduler: Missing parents: List()
24/03/13 20:48:22 INFO DAGScheduler: Submitting ShuffleMapStage 4 (MapPartitionsRDD[14] at count at cell9.sc:1), which has no missing parents
24/03/13 20:48:22 INFO MemoryStore: Block broadcast_3 stored as values in memory (estimated size 15.1 KiB, free 4.5 GiB)
24/03/13 20:48:22 INFO MemoryStore: Block broadcast_3_piece0 stored as bytes in memory (estimated size 7.7 KiB, free 4.5 GiB)
24/03/13 20:48:22 INFO BlockManagerInfo: Added broadcast_3_piece0 in memory on ubuntu:45869 (size: 7.7 KiB, free: 4.5 GiB)
24/03/13 20:48:22 INFO SparkContext: Created broadcast 3 from br

count = 30003
num partitions = 10


In [10]:
df103.rdd.mapPartitionsWithIndex { (p,i) => List((p, i.length)).iterator }.collect()

24/03/13 20:48:22 INFO SparkContext: Starting job: collect at cell10.sc:1
24/03/13 20:48:22 INFO DAGScheduler: Got job 5 (collect at cell10.sc:1) with 10 output partitions
24/03/13 20:48:22 INFO DAGScheduler: Final stage: ResultStage 7 (collect at cell10.sc:1)
24/03/13 20:48:22 INFO DAGScheduler: Parents of final stage: List()
24/03/13 20:48:22 INFO DAGScheduler: Missing parents: List()
24/03/13 20:48:22 INFO DAGScheduler: Submitting ResultStage 7 (MapPartitionsRDD[23] at mapPartitionsWithIndex at cell10.sc:1), which has no missing parents
24/03/13 20:48:22 INFO MemoryStore: Block broadcast_5 stored as values in memory (estimated size 21.9 KiB, free 4.5 GiB)
24/03/13 20:48:22 INFO MemoryStore: Block broadcast_5_piece0 stored as bytes in memory (estimated size 9.9 KiB, free 4.5 GiB)
24/03/13 20:48:22 INFO BlockManagerInfo: Added broadcast_5_piece0 in memory on ubuntu:45869 (size: 9.9 KiB, free: 4.5 GiB)
24/03/13 20:48:22 INFO SparkContext: Created broadcast 5 from broadcast at DAGSchedu

res10: Array[(Int, Int)] = Array(
  (0, 1299),
  (1, 300),
  (2, 300),
  (3, 300),
  (4, 300),
  (5, 300),
  (6, 300),
  (7, 300),
  (8, 300),
  (9, 26304)
)